In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import pandas_ta as ta
import os

In [3]:
def get_indicators(symbol, start, end):

    # Indikatoren festlegen
    momentum_indicators = ["rsi", "macd", "stoch", "mom", "tsi", "adx"]
    volume_indicators = ["obv", "vwap", "pvo", "ad", "mfi", "cmf"]
    volatility_indicators = ["bbands", "atr", "kc"] # Ichimoku separat wegen potenziellem Data Leakage
    candlestick_patterns = ["hammer", "morningstar", "hangingman", "darkcloudcover", "engulfing", "doji"] # Patterns "3whitesoldiers" und "3blackcrows" sind zu selten, entfernt

    # Stock Data laden
    df = yf.download(symbol, start, end)

    # Indikatoren berechnen
    for momentum_indicator in momentum_indicators:
        df.ta(kind=momentum_indicator, append=True)
    
    for volume_indicator in volume_indicators:
        df.ta(kind=volume_indicator, append=True)
    
    for volatility_indicator in volatility_indicators:
        df.ta(kind=volatility_indicator, append=True)

    # Separat wegen Sub-Indikator mit potenziellem Data Leakage: 'ICS_26'
    ichimoku = ta.ichimoku(high=df['High'], low=df['Low'], close=df['Close'], lookahead=False)
    ichimoku_visible = ichimoku[0]
    df = pd.concat([df, ichimoku_visible], axis=1)

    # Candlestick Patterns
    patterns_df = df.ta.cdl_pattern(name=candlestick_patterns) #cdl_pattern generiert separates df
    df = pd.concat([df, patterns_df], axis=1)

    # Binary Encoding der Candlestick Patterns
    bullish_patterns = ['CDL_HAMMER', 'CDL_MORNINGSTAR'] # Werte sind 0 oder 100
    bearish_patterns= ['CDL_HANGINGMAN', 'CDL_DARKCLOUDCOVER'] # Werte sind 0 oder -100
    bull_bear_patterns = ['CDL_ENGULFING'] # Werte sind -100, 0, 100
    continuation_patterns = ['CDL_DOJI_10_0.1'] # Werte sind 0 oder 100
    candlestick_columns = bullish_patterns + bearish_patterns + bull_bear_patterns + continuation_patterns

    for bullish_pattern in bullish_patterns:
        df[f'{bullish_pattern}_bullish'] = (df[bullish_pattern] == 100).astype(int)

    for bearish_pattern in bearish_patterns:
        df[f'{bearish_pattern}_bearish'] = (df[bearish_pattern] == -100).astype(int)

    for bull_bear_pattern in bull_bear_patterns:
        df[f'{bull_bear_pattern}_bullish'] = (df[bull_bear_pattern] == 100).astype(int)
        df[f'{bull_bear_pattern}_bearish'] = (df[bull_bear_pattern] == -100).astype(int)

    for continuation_pattern in continuation_patterns:
        df[f'{continuation_pattern}_continuation'] = (df[continuation_pattern] == 100).astype(int)

    df.drop(columns=candlestick_columns, inplace=True) # ursprüngliche Candlestick Spalten nicht mehr nötig


    return df


def add_interest(df):
    interest_path = os.path.join('data', 'DFF.csv')
    interest_df = pd.read_csv(interest_path, sep=',', encoding='UTF-8', parse_dates=['DATE'], index_col='DATE')
    df = pd.merge(df, interest_df, left_index=True, right_index=True) # Left-Join auf dem Index <-- Datum

    return df


def add_vix(df):
    vix_path = os.path.join('data','VIX_History.csv')
    vix_df = pd.read_csv(vix_path, sep=',', encoding='UTF-8', parse_dates=['DATE'], index_col='DATE')

    vix_column_renaming = {
        'OPEN':'VIX_open',
        'CLOSE':'VIX_close',
        'HIGH':'VIX_high',
        'LOW':'VIX_low'
    }
    vix_df.rename(columns=vix_column_renaming, inplace=True)

    # Berechnung absolute und relative Veränderung
    vix_df['VIX_abs_change'] = vix_df['VIX_close'] - vix_df['VIX_open']
    vix_df['VIX_rel_change'] = vix_df['VIX_abs_change'] / vix_df['VIX_open']

    df = pd.merge(df, vix_df, left_index=True, right_index=True)

    return df


def add_s_p_500(df, start, end):
    gspc_df = yf.download('^GSPC', start, end)
    gspc_df.drop(columns=['Adj Close', 'Volume'], inplace=True)
    gspc_column_renaming = {
        'Open':'S_P_500_open',
        'Close':'S_P_500_close',
        'High':'S_P_500_high',
        'Low':'S_P_500_low'
    }
    gspc_df.rename(columns=gspc_column_renaming, inplace=True)

    # Berechnung absolute und relative Veränderung
    gspc_df['S_P_500_abs_change'] = gspc_df['S_P_500_close'] - gspc_df['S_P_500_open']
    gspc_df['S_P_rel_change'] = gspc_df['S_P_500_abs_change'] / gspc_df['S_P_500_open']

    df = pd.merge(df, gspc_df, left_index=True, right_index=True)

    return df


def add_option_volume(df, stock):

    stock = stock.replace('-', '') # Beispiel BRK-B -> BRKB

    #######################
    #       Monthly       #
    #######################

    # von 2003 bis Ende 2006 ist nur das durchschnittliche tägliche Optionsvolumen jedes Monats verfügbar
    
    # DataFrame erstellen, wo für jeden Monat das tägliche Durchschnittsvolumen in jeden Tag geschrieben wird
    date_range = pd.date_range(start='2003-01-01', end='2006-12-31')
    monthly_option_df = pd.DataFrame(index=date_range, columns=['Option Volume'])

    options_path_monthly =os.path.join('data', 'options', 'monthly')

    for year in range(2003, 2006+1):
        year_str = str(year)

        for month in range(1, 12+1):

            month_str = str(month).zfill(2) # Null auffüllen wo nötig, beispiel '1' -> '01'

            if year < 2013:  # Datenformat von 2003 bis 2012 ist .xls, danach .xlsx
                file_extension = '.xls'
            else:
                file_extension = '.xlsx'
            
            file_name = f'{year}_{month_str}_rank_wosym{file_extension}'
            specific_path = os.path.join(options_path_monthly, file_name)

            options_df = pd.read_excel(specific_path) # Dependencies neben Pandas: openpyxl und xlrd <-- pip install

            # Spalten werden anfangs klein geschrieben...
            if year == 2003 or (year == 2004 and month in [1,2,3]):
                symbol_column = 'symbol'
            else:
                symbol_column = 'Symbol'

            average_daily_volume = options_df.loc[options_df[symbol_column] == stock, 'Tot ADV'].values

            if len(average_daily_volume) > 0:  # Check if data exists for the stock in that month
                # Fill 'Option Volume' column for the month with the extracted value
                start_date = pd.Timestamp(f'{year}-{month_str}-01')
                end_date = pd.Timestamp(start_date.year, start_date.month, start_date.days_in_month)
                monthly_option_df.loc[start_date:end_date, 'Option Volume'] = average_daily_volume[0]


    #######################
    #        Daily        #
    #######################

    # Tägliches Optionsvolumen ab 2007 verfügbar
    yearly_dfs = []
    options_path_yearly = os.path.join('data', 'options', 'daily')

    for year in range(2007, 2022+1):
        year_str = str(year)
        specific_path = os.path.join(options_path_yearly, f'daily_volume_{year_str}.csv')

        options_df = pd.read_csv(specific_path, sep=',', encoding='UTF-8', parse_dates=['Trade Date'])

        stock_df = options_df[options_df['Underlying'] == stock]

        # Nach Datum gruppieren und anschliessend das Volumen der gruppierten Zeilen summieren
        daily_volume_sum = stock_df.groupby('Trade Date')['Average Daily Volume'].sum().reset_index()

        daily_volume_sum.rename(columns={'Average Daily Volume':'Option Volume'}, inplace=True)

        yearly_dfs.append(daily_volume_sum)

    daily_option_df = pd.concat(yearly_dfs, ignore_index=True)
    daily_option_df.set_index('Trade Date', inplace=True)



    #######################
    #       Zusammen      #
    #######################

    option_df = pd.concat([monthly_option_df, daily_option_df])

    df = pd.merge(df, option_df, left_index=True, right_index=True)

    return df


def add_yfinance(df, stock):
    ticker = yf.Ticker(stock)
    df['Symbol'] = stock
    df['Sector'] = ticker.info['sector']
    df['Industry'] = ticker.info['industry']

    return df


def add_change(df):
    df['Absolute change'] = df['Close'] - df['Open']
    df['Relative change'] = df['Absolute change'] / df['Open']

    df.drop(columns=['Adj Close'], inplace=True)

    return df


def rearrange_columns(df):
    columns = df.columns.tolist()

    # Remove 'Absolute change' and 'Relative change' from their original positions
    columns.remove('Absolute change')
    columns.remove('Relative change')

    # Insert 'Absolute change' and 'Relative change' at the 5th and 6th positions respectively
    columns.insert(4, 'Absolute change')
    columns.insert(5, 'Relative change')

    # Reorder the columns in the DataFrame
    df = df[columns]
    return df


def add_next_close(df):
    df['Next close'] = df['Close'].shift(-1) # Zielvariable = Close des nächsten Tages
    df.drop(df.tail(1).index, inplace=True) # Letze Zeile hat bei 'Next Close' NaN -> droppen
    return df


def save_df(df, stock, option_volume):

    if option_volume:
        folder_path = os.path.join('data', 'stock_dataframes_OV')
    
    else:
        folder_path = os.path.join('data', 'stock_dataframes')

    # Falls Ordner noch nicht besteht, erstellen
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)    

    file_path = os.path.join(folder_path, f'{stock}.csv')
    df[77:].to_csv(file_path, index=True, index_label='Date') # Ab Zeile 77 wegen Einschwingphase einiger Indikatoren
    

def get_s_p_stocks(threshold):

    s_p_path = os.path.join('data', 'sp500_companies.csv')
    s_p_df = pd.read_csv(s_p_path, sep=',', encoding='UTF-8')
    s_p_reduced = s_p_df[['Symbol', 'Weight']]

    if threshold == 100:
        relevant_stocks = s_p_reduced['Symbol'].tolist()
        return relevant_stocks

    weight_sum = 0
    relevant_stocks = []

    for i in range(s_p_reduced.shape[0]):
        if weight_sum*100 >= threshold:
            return relevant_stocks

        weight_sum += s_p_reduced.iloc[i]['Weight']
        relevant_stocks.append(s_p_reduced.iloc[i]['Symbol'])

    return 'Something went wrong'


def prepare_features(stock_symbol, option_volume=True):
    end = datetime.strptime('2023-01-01', '%Y-%m-%d')
    start = datetime(end.year - 20, end.month, end.day)

    df = get_indicators(stock_symbol, start, end)
    df = add_interest(df)
    df = add_vix(df)
    df = add_s_p_500(df, start, end)
    if option_volume:
        df = add_option_volume(df, stock_symbol)
    df = add_yfinance(df, stock_symbol)
    df = add_change(df)
    df = rearrange_columns(df)
    df = add_next_close(df)
    save_df(df, stock_symbol, option_volume)

    print(f'DataFrame for {stock_symbol} has been prepared and saved')


def combine_stocks(save_file_name='combined_dataframe', return_df=False):
    relevant_stocks = ['MSFT', 'AAPL', 'NVDA', 'GOOG', 'AMZN', 'BRK-B', 'LLY', 'JPM', 'XOM', 'WMT', 'UNH', 'MA', 'PG', 'JNJ', 'COST', 'HD', 'MRK', 'ORCL', 'CVX', 'BAC', 'KO', 'CRM', 'NFLX', 'PEP', 'AMD', 'TMO', 'ADBE', 'WFC', 'LIN', 'QCOM', 'CSCO', 'MCD', 'ACN', 'DIS', 'DHR', 'ABT', 'INTU', 'GE', 'CAT', 'AMAT', 'AXP', 'TXN', 'VZ', 'AMGN', 'PFE', 'MS', 'CMCSA', 'IBM', 'NEE', 'UNP']

    stock_df_path = os.path.join('data', 'stock_dataframes')

    all_stocks = []

    for symbol in relevant_stocks:
        file_path = os.path.join(stock_df_path, f'{symbol}.csv')
        stock_df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
        all_stocks.append(stock_df)

    # Alle DataFrames zusammenführen
    big_df = pd.concat(all_stocks)

    print('Checking for NaN Values...')
    for column in big_df.columns:
        nan_sum = big_df[column].isna().sum()
        if nan_sum > 0:
            print(f'Column {column} has {nan_sum} NaN Values')

    old_shape = big_df.shape
    big_df.dropna(subset=['VWAP_D'], inplace=True)
    print(f'Dropped {old_shape[0] - big_df.shape[0]} rows. (1 is the bugged AMD row)')

    # Nach Datum sortieren
    # big_df.sort_values(by=['Date'], inplace=True)
    big_df.sort_values(by=['Date', 'Symbol'], inplace=True)

    # One-Hot Encoding
    # 10 Sektoren zu Spalten. Die 30 Sub-Sektoren ('Industry') wären zu viel
    one_hot_encoded = pd.get_dummies(big_df['Sector'], prefix='Sector')
    one_hot_encoded_numeric = one_hot_encoded.astype(int)
    big_df = pd.concat([big_df, one_hot_encoded_numeric], axis=1)
    big_df.drop(columns=['Sector', 'Industry'], inplace=True)

    # Zielvariable 'Next close' als letzte Spalte
    column_list = big_df.columns.tolist()
    column_list.remove('Next close')
    column_list.append('Next close')
    big_df = big_df[column_list]

    save_path = os.path.join('data', 'ML_data')
    # Falls Ordner noch nicht besteht, erstellen
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_file_path = os.path.join(save_path, f'{save_file_name}.csv')
    big_df.to_csv(save_file_path, index=True)
    print(f'Saved DataFrame as {save_file_name}.csv at path {save_file_path}')

    if return_df:
        return big_df



In [3]:
end = datetime.strptime('2023-01-01', '%Y-%m-%d')
start = datetime(end.year - 20, end.month, end.day)

aapl_df = get_indicators('AAPL', start, end)
aapl_df = add_interest(aapl_df)
aapl_df = add_vix(aapl_df)
aapl_df = add_s_p_500(aapl_df, start, end)
aapl_df = add_option_volume(aapl_df, 'AAPL')
aapl_df = add_yfinance(aapl_df, 'AAPL')
aapl_df = add_change(aapl_df)
aapl_df = rearrange_columns(aapl_df)
aapl_df = add_next_close(aapl_df)
# save_df(aapl_df, 'AAPL')

aapl_df.head(1010)
# aapl_df[['Close', 'Next close']].tail(1010)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Absolute change,Relative change,Volume,RSI_14,MACD_12_26_9,MACDh_12_26_9,...,S_P_500_high,S_P_500_low,S_P_500_close,S_P_500_abs_change,S_P_rel_change,Option Volume,Symbol,Sector,Industry,Next close
2003-01-02,0.256429,0.266429,0.256250,0.264286,0.007857,0.030640,181428800,NaN,NaN,NaN,...,909.030029,879.820007,909.030029,29.210022,0.033200,1133.619048,AAPL,Technology,Consumer Electronics,0.266071
2003-01-03,0.264286,0.266607,0.260536,0.266071,0.001785,0.006754,147453600,NaN,NaN,NaN,...,911.250000,903.070007,908.590027,-0.440002,-0.000484,1133.619048,AAPL,Technology,Consumer Electronics,0.266071
2003-01-06,0.268393,0.274643,0.265714,0.266071,-0.002322,-0.008652,390532800,NaN,NaN,NaN,...,931.770020,908.590027,929.010010,20.419983,0.022474,1133.619048,AAPL,Technology,Consumer Electronics,0.265179
2003-01-07,0.264107,0.267857,0.258393,0.265179,0.001072,0.004059,342344800,NaN,NaN,NaN,...,930.809998,919.929993,922.929993,-6.080017,-0.006545,1133.619048,AAPL,Technology,Consumer Electronics,0.259821
2003-01-08,0.260357,0.262679,0.257857,0.259821,-0.000536,-0.002059,229644800,NaN,NaN,NaN,...,922.929993,908.320007,909.929993,-13.000000,-0.014086,1133.619048,AAPL,Technology,Consumer Electronics,0.262143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-12-29,2.998214,3.050000,2.977143,3.030000,0.031786,0.010602,1076429200,48.895450,-0.035800,-0.026991,...,1427.000000,1416.839966,1418.300049,-6.409912,-0.004499,47047.9,AAPL,Technology,Consumer Electronics,2.992857
2007-01-03,3.081786,3.092143,2.925000,2.992857,-0.088929,-0.028856,1238319600,46.073142,-0.034354,-0.020436,...,1429.420044,1407.859985,1416.599976,-1.430054,-0.001008,70418,AAPL,Technology,Consumer Electronics,3.059286
2007-01-04,3.001786,3.069643,2.993571,3.059286,0.057500,0.019155,847260400,51.468565,-0.027530,-0.010890,...,1421.839966,1408.430054,1418.339966,1.739990,0.001228,41805,AAPL,Technology,Consumer Electronics,3.037500
2007-01-05,3.063214,3.078571,3.014286,3.037500,-0.025714,-0.008395,834741600,49.711902,-0.023608,-0.005574,...,1418.339966,1405.750000,1409.709961,-8.630005,-0.006085,42297,AAPL,Technology,Consumer Electronics,3.052500


In [4]:
aapl_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Absolute change', 'Relative change',
       'Volume', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
       'STOCHk_14_3_3', 'STOCHd_14_3_3', 'MOM_10', 'TSI_13_25_13',
       'TSIs_13_25_13', 'ADX_14', 'DMP_14', 'DMN_14', 'OBV', 'VWAP_D',
       'PVO_12_26_9', 'PVOh_12_26_9', 'PVOs_12_26_9', 'AD', 'MFI_14', 'CMF_20',
       'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', 'BBB_5_2.0', 'BBP_5_2.0',
       'ATRr_14', 'KCLe_20_2', 'KCBe_20_2', 'KCUe_20_2', 'ISA_9', 'ISB_26',
       'ITS_9', 'IKS_26', 'CDL_HAMMER_bullish', 'CDL_MORNINGSTAR_bullish',
       'CDL_HANGINGMAN_bearish', 'CDL_DARKCLOUDCOVER_bearish',
       'CDL_ENGULFING_bullish', 'CDL_ENGULFING_bearish',
       'CDL_DOJI_10_0.1_continuation', 'DFF', 'VIX_open', 'VIX_high',
       'VIX_low', 'VIX_close', 'VIX_abs_change', 'VIX_rel_change',
       'S_P_500_open', 'S_P_500_high', 'S_P_500_low', 'S_P_500_close',
       'S_P_500_abs_change', 'S_P_rel_change', 'Option Volume', 'Symbol',
   

In [5]:
aapl_df.shape

(5033, 65)

In [6]:
for column in aapl_df.columns:
    print(column, aapl_df[column].isna().sum())

Open 0
High 0
Low 0
Close 0
Absolute change 0
Relative change 0
Volume 0
RSI_14 14
MACD_12_26_9 33
MACDh_12_26_9 33
MACDs_12_26_9 33
STOCHk_14_3_3 15
STOCHd_14_3_3 17
MOM_10 10
TSI_13_25_13 37
TSIs_13_25_13 49
ADX_14 27
DMP_14 14
DMN_14 14
OBV 0
VWAP_D 0
PVO_12_26_9 25
PVOh_12_26_9 33
PVOs_12_26_9 33
AD 0
MFI_14 14
CMF_20 19
BBL_5_2.0 4
BBM_5_2.0 4
BBU_5_2.0 4
BBB_5_2.0 4
BBP_5_2.0 4
ATRr_14 14
KCLe_20_2 20
KCBe_20_2 19
KCUe_20_2 20
ISA_9 51
ISB_26 77
ITS_9 8
IKS_26 25
CDL_HAMMER_bullish 0
CDL_MORNINGSTAR_bullish 0
CDL_HANGINGMAN_bearish 0
CDL_DARKCLOUDCOVER_bearish 0
CDL_ENGULFING_bullish 0
CDL_ENGULFING_bearish 0
CDL_DOJI_10_0.1_continuation 0
DFF 0
VIX_open 0
VIX_high 0
VIX_low 0
VIX_close 0
VIX_abs_change 0
VIX_rel_change 0
S_P_500_open 0
S_P_500_high 0
S_P_500_low 0
S_P_500_close 0
S_P_500_abs_change 0
S_P_rel_change 0
Option Volume 0
Symbol 0
Sector 0
Industry 0
Next close 0


In [7]:
prepare_features('AAPL', option_volume=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


DataFrame for AAPL has been prepared and saved


In [8]:
selected_stocks = get_s_p_stocks(60)
selected_stocks
# len(selected_stocks)

['MSFT',
 'AAPL',
 'NVDA',
 'GOOG',
 'GOOGL',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'TSLA',
 'XOM',
 'WMT',
 'UNH',
 'MA',
 'PG',
 'JNJ',
 'COST',
 'HD',
 'MRK',
 'ORCL',
 'CVX',
 'BAC',
 'ABBV',
 'KO',
 'CRM',
 'NFLX',
 'PEP',
 'AMD',
 'TMO',
 'ADBE',
 'WFC',
 'LIN',
 'QCOM',
 'CSCO',
 'MCD',
 'ACN',
 'DIS',
 'TMUS',
 'DHR',
 'ABT',
 'INTU',
 'GE',
 'CAT',
 'AMAT',
 'AXP',
 'TXN',
 'VZ']

In [9]:
selected_stocks = get_s_p_stocks(60)

for stock_symbol in selected_stocks:
    prepare_features(stock_symbol, option_volume=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

DataFrame for MSFT has been prepared and saved



[*********************100%%**********************]  1 of 1 completed


DataFrame for AAPL has been prepared and saved


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

...

DataFrame for AXP has been prepared and saved


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


DataFrame for TXN has been prepared and saved


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


DataFrame for VZ has been prepared and saved


In [10]:
# Zusätzliche Aktien
additonal_stocks = ['AMGN', 'PFE', 'MS', 'CMCSA', 'IBM', 'NEE', 'UNP']

for stock_symbol in additonal_stocks:
    prepare_features(stock_symbol, option_volume=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


DataFrame for AMGN has been prepared and saved


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


DataFrame for PFE has been prepared and saved


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

DataFrame for IBM has been prepared and saved


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


DataFrame for NEE has been prepared and saved


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


DataFrame for UNP has been prepared and saved


In [11]:
symbols = get_s_p_stocks(60)
additional_symbols = ['AMGN', 'PFE', 'MS', 'CMCSA', 'IBM', 'NEE', 'UNP']

all_symbols = symbols + additional_symbols

nan_dict = {'Symbol': [], 'Column': [], 'NaN_Count': []}
rows_dict = {'Symbol':[], 'Rows': []}

stock_df_path = os.path.join('data', 'stock_dataframes')

for symbol in all_symbols:
    # jedes DataFrame lesen
    file_path = os.path.join(stock_df_path, f'{symbol}.csv')
    stock_df = pd.read_csv(file_path, index_col='Date', parse_dates=True)

    # NaN Werte auslesen und in DF schreiben
    for column in stock_df.columns:
        nan_number = stock_df[column].isna().sum()
        if nan_number > 0:
            nan_dict['Symbol'].append(symbol)
            nan_dict['Column'].append(column)
            nan_dict['NaN_Count'].append(nan_number)
    
    rows_dict['Symbol'].append(symbol)
    rows_dict['Rows'].append(stock_df.shape[0])

nan_df = pd.DataFrame(nan_dict)

rows_df = pd.DataFrame(rows_dict)

nan_df

,Symbol,Column,NaN_Count
0,AMD,VWAP_D,1


In [12]:
rows_df

,Symbol,Rows
0,MSFT,4956
1,AAPL,4956
2,NVDA,4956
3,GOOG,4546
4,GOOGL,4546
5,AMZN,4956
6,META,2595
7,BRK-B,4956
8,LLY,4956
9,AVGO,3297


In [13]:
valid_stocks = rows_df[rows_df['Rows'] >= 4000]['Symbol'].tolist()
valid_stocks.remove('GOOGL')

print(len(valid_stocks))
valid_stocks

50


['MSFT',
 'AAPL',
 'NVDA',
 'GOOG',
 'AMZN',
 'BRK-B',
 'LLY',
 'JPM',
 'XOM',
 'WMT',
 'UNH',
 'MA',
 'PG',
 'JNJ',
 'COST',
 'HD',
 'MRK',
 'ORCL',
 'CVX',
 'BAC',
 'KO',
 'CRM',
 'NFLX',
 'PEP',
 'AMD',
 'TMO',
 'ADBE',
 'WFC',
 'LIN',
 'QCOM',
 'CSCO',
 'MCD',
 'ACN',
 'DIS',
 'DHR',
 'ABT',
 'INTU',
 'GE',
 'CAT',
 'AMAT',
 'AXP',
 'TXN',
 'VZ',
 'AMGN',
 'PFE',
 'MS',
 'CMCSA',
 'IBM',
 'NEE',
 'UNP']

In [4]:
combine_stocks()

Checking for NaN Values...
Column VWAP_D has 1 NaN Values
Dropped 1 rows. (1 is the bugged AMD row)
Saved DataFrame as combined_dataframe.csv at path data\ML_data\combined_dataframe.csv
